# Instalar as dependências necessárias

In [1]:
!pip install -U openai-whisper
!pip install gTTS
!apt install ffmpeg
! pip install sounddevice wavio
! pip install ipywebrtc notebook
!pip install groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 17.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s

# Importar as bibliotecas

In [2]:
import whisper
from gtts import gTTS
import os
from google.colab import drive, output
from IPython.display import Audio, display
import time
from ipywebrtc import AudioRecorder, CameraStream
import ipywidgets as widgets
from groq import Groq

In [3]:
model = whisper.load_model("base")

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 126MiB/s]


# Configuração API da GROQ usando modelo deepseek
Modelo: deepseek-r1-distill-qwen-32b

In [4]:
GROQ_API_KEY = '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'

if not os.getenv("GROQ_API_KEY"):
    os.environ["GROQ_API_KEY"] = GROQ_API_KEY

client = Groq()

# Gravação pelo colab

First, we need to enable some Colab widgets so that we can make an audio recording.

In [5]:
output.enable_custom_widget_manager()

Press the circle button and start speaking. It may not look it, but the widget will be capturing sound. Click the circle button again when you are finished. The widget will immediately begin to play back what it captured.

In [24]:
camera = CameraStream(constraints={'audio': True,'video':False})
recorder = AudioRecorder(stream=camera)
recorder

AudioRecorder(audio=Audio(value=b'', format='webm'), stream=CameraStream(constraints={'audio': True, 'video': …

The audio format captured above is not readable by PyTorch. In this step, we convert our recording into a format that PyTorch can understand.

In [29]:
with open('recording.webm', 'wb') as f:
    f.write(recorder.audio.value)
!ffmpeg -i recording.webm -ac 1 -f wav my_recording.wav -y -hide_banner -loglevel panic

# Transcrever o áudio

In [30]:
audio_path = "/content/my_recording.wav"
result = model.transcribe(audio_path)
transcription = result["text"]
print("Transcrição:", transcription)

/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcrição:  Por favor, me expliquem poucas palavras, a diferença entre o quadrado e o cubo.


# Comunicação com o modelo deepseek

In [31]:
completion = client.chat.completions.create(
    model="deepseek-r1-distill-qwen-32b",
    messages=[
        {
            "role": "user",
            "content": transcription
        }
    ]
)
print(completion.choices[0].message.content)

<think>
Primeiro, entendo que o usuário deseja compreender a diferença entre "quadrado" e "cubo" de maneira simples e direta.

Em termos matemáticos, o quadrado de um número é o resultado da multiplicação desse número por ele mesmo. Por exemplo, 2 elevado ao quadrado é 2 x 2, que resulta em 4.

Já o cubo de um número é obtido multiplicando o número por ele mesmo duas vezes. Assim, 2 elevado ao cubo é 2 x 2 x 2, resultando em 8.

Portanto, a principal diferença está na quantidade de vezes que o número é multiplicado por si mesmo: duas vezes para o quadrado e três vezes para o cubo.
</think>

Claro! Vamos entender a diferença entre o quadrado e o cubo de um número de forma simples.

### **Quadrado de um Número**
O **quadrado** de um número é o resultado da multiplicação desse número por ele mesmo.

- **Fórmula:**  
  \[
  \text{Quadrado de } a = a^2 = a \times a
  \]

- **Exemplo:**  
  \[
  2^2 = 2 \times 2 = 4
  \]

### **Cubo de um Número**
O **cubo** de um número é o resultado da mul

# Sintetizar a fala em português

In [32]:
# Usar gTTS para sintetizar a fala em português
audio_path = 'output.mp3'
tts = gTTS(text=completion.choices[0].message.content, lang='pt')
tts.save(audio_path)

# Reproduzir o áudio sintetizado
Audio(audio_path)

Output hidden; open in https://colab.research.google.com to view.